# 要約

### ソフトウェアのインストール

sumy, tinysegmenter
```
conda install -c conda-forge sumy tinysegmenter
```
- https://anaconda.org/conda-forge/sumy
- https://anaconda.org/conda-forge/tinysegmenter

### テキストデータ

青空文庫　北大路魯山人「料理の秘訣」<br>
https://www.aozora.gr.jp/cards/001403/card54988.html
- テキストファイル 54988_ruby_50511.zip をダウンロード
- 最初と最後の本文以外を削除

参考
- https://www.inoue-kobo.com/ai_ml/sumy-aozora/index.html
- http://maruo51.com/2019/07/07/python-sumy/

In [ ]:
import pandas as pd
import re
import spacy

file = 'data/ryorino_hiketsu.txt'

with open(file, 'r', encoding='shift_jis') as f:
    text = f.read()

# ルビ、注釈を削除
text = re.sub(r'《.+?》', '', text)
text = re.sub(r'［＃.+?］', '', text)
text = re.sub(r'\n', '', text)

### テキストの準備

In [ ]:
# 日本語モデル
nlp = spacy.load('ja_core_news_lg')

# 1行ずつ、分かち書きに変換
corpus = []
original = []
doc = nlp(text)
for s in doc.sents:
    original.append(s)
    tokens = []
    for t in s:
        if t.pos_ not in ['SPACE']:
            tokens.append(t.lemma_)
    corpus.append(' '.join(tokens))

# サイズの確認
print(len(corpus))
print(len(original))

### 要約の作成

- 頻出単語を含む文を選択
- 文を作成しているわけではない

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# corpusをトークン化
parser = PlaintextParser.from_string(''.join(corpus), Tokenizer('japanese'))

# 初期化
summarizer = LexRankSummarizer()
# 空白を除外
summarizer.stop_words = [' ']

# 要約
# - sentences_count=3: 3行に要約
summary = summarizer(document=parser.document, sentences_count=3)

# 元の文を表示
# - 見出し語 (lemma) の分かち書きから、元の文を検索して表示
for sentence in summary:
    print(original[corpus.index(sentence.__str__())])

<hr>

注意
- 文章によってうまくいくものと、いかないものがある
- 上記は、いくつか試したなかで、うまくいったもの